In [1]:
# ProcessIdentification class
# Necessary libraries
import numpy as np
import os
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
he_init = tf.variance_scaling_initializer()
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")

class SecondOrderProcessIdentification:
    def __init__(self, n_hidden_layers=2, n_neurons=100, optimizer_class=tf.train.MomentumOptimizer,
                 momentum = 0.9, learning_rate=0.01, n_inputs = 7, n_outputs = 1, batch_size=50, activation=tf.nn.relu,
                 initializer = None, learning_object=None, loaded_data=None, random_state=None):
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.optimizer_class = optimizer_class
        self.momentum = momentum
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.activation = activation
        self.initializer = initializer        
        self.learning_object = learning_object
        self.loaded_data = loaded_data
        self.random_state = random_state
        self._session = None
        
    def data_preparation(self):
        train_set, test_set = train_test_split(self.loaded_data, test_size=0.2, random_state=42)
        X_train, y_train = train_set.iloc[:,:7].values, train_set.iloc[:,7:].values
        X_test, y_test = test_set.iloc[:,:7].values, test_set.iloc[:,7:].values
        X_scale_minmax = X_train
        scaler = MinMaxScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)   
        return X_train, y_train, X_test, y_test, X_scale_minmax
    
    def data_preparation_theta(self):
        train_set, test_set = train_test_split(self.loaded_data, test_size=0.2, random_state=42)
        X_train, y_train = train_set.iloc[:,:17].values, train_set.iloc[:,17:].values
        X_test, y_test = test_set.iloc[:,:17].values, test_set.iloc[:,17:].values
        X_scale_minmax = X_train
        scaler = MinMaxScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)   
        return X_train, y_train, X_test, y_test, X_scale_minmax
    
    def operating_data_preparation(self, X_scale_minmax):
        X_op, y_op = self.loaded_data.iloc[:,:7].values, self.loaded_data.iloc[:,7:].values
        process_gain = X_op[0,:]        
        scaler = MinMaxScaler()
        scaler.fit(X_scale_minmax)
        X_op = scaler.transform(X_op)
        return X_op, y_op, process_gain
    
    def operating_data_preparation_theta(self, X_scale_minmax):
        X_op, y_op = self.loaded_data.iloc[:,:17].values, self.loaded_data.iloc[:,17:].values
        scaler = MinMaxScaler()
        scaler.fit(X_scale_minmax)
        X_op = scaler.transform(X_op)
        return X_op, y_op

    def _dnn(self, inputs):
        for layer in range(self.n_hidden_layers):
            inputs = tf.layers.dense(inputs, self.n_neurons, kernel_initializer=self.initializer,
                                     name="hidden%d" % (layer + 1))
            inputs = self.activation(inputs, name="hidden%d_out" % (layer + 1))
        return inputs
    
    def _build_graph(self, n_inputs, n_outputs):
        if self.random_state is not None:
            tf.set_random_seed(self.random_state)
            np.random.seed(self.random_state)
            
        X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
        y = tf.placeholder(tf.float32, shape=(None), name="y")
        
        dnn_outputs = self._dnn(X)
        
        y_predict = tf.layers.dense(dnn_outputs, n_outputs, name="outputs")
        
        RMSE = tf.sqrt(tf.losses.mean_squared_error(y, y_predict), name="loss")
        
        optimizer = self.optimizer_class(learning_rate=self.learning_rate, momentum=self.momentum, use_nesterov=True)
        training_op = optimizer.minimize(RMSE)
        
        accuracy = tf.reduce_mean(1-(abs(y_predict - y)/y), name="accuracy")
        
        init = tf.global_variables_initializer()
        saver = tf.train.Saver()
        
        self._X, self._y = X, y
        self._y_predict, self._RMSE = y_predict, RMSE
        self._training_op, self._accuracy = training_op, accuracy
        self._init, self._saver = init, saver

    def close_session(self):
        if self._session:
            self._session.close()

    def fit(self, X, y, n_epochs=20):
        self.close_session()
        
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(self.n_inputs, self.n_outputs)        
        
        self._session = tf.Session(graph=self._graph)
        
        root_logdir = "tf_logs_2nd_%s_RMSE" % (self.learning_object)
        logdir = "{}/run-{}/".format(root_logdir, now)
        RMSE_summary = tf.summary.scalar('RMSE', self._RMSE)
        file_writer = tf.summary.FileWriter(logdir, self._session.graph)
        
        with self._session.as_default() as sess:
            self._init.run()
            for epoch in range(n_epochs):
                rnd_idx = np.random.permutation(len(X))
                for rnd_indices in np.array_split(rnd_idx, len(X) // self.batch_size):
                    X_batch, y_batch = X[rnd_indices], y[rnd_indices]
                    feed_dict = {self._X: X_batch, self._y: y_batch}
                    sess.run(self._training_op, feed_dict=feed_dict)
                acc_batch = self._accuracy.eval(feed_dict=feed_dict)
                RMSE_batch = self._RMSE.eval(feed_dict=feed_dict)
                summary_str = RMSE_summary.eval(feed_dict=feed_dict)
                file_writer.add_summary(summary_str, epoch)
                print("{}\tbatch data accuracy: {:.4f}\tbatch data RMSE: {:.6f}".format(epoch, acc_batch, RMSE_batch))
            
            save_path = self._saver.save(sess, "./2nd_order_process_%s_learning.ckpt" % self.learning_object)
            file_writer.close()
            
    def predict(self, X, y, learning_object=None):
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(self.n_inputs, self.n_outputs)
            
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:
            self._saver.restore(sess, "./2nd_order_process_%s_learning.ckpt" % self.learning_object)
            accuracy_val = self._accuracy.eval(feed_dict={self._X: X, self._y: y})
            X_new_scaled = X[:10]
            y_pred = self._y_predict.eval(feed_dict={self._X: X_new_scaled})
            
            print("\n")
            print("%s test data accuracy:" % self.learning_object, accuracy_val)
            print("\n")
        
        return y_pred
    
    def operating_predict(self, X, y, learning_object=None):
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(self.n_inputs, self.n_outputs)
            
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:
            self._saver.restore(sess, "./2nd_order_process_%s_learning.ckpt" % self.learning_object)
            X_new_scaled = X[:1]
            y_pred = self._y_predict.eval(feed_dict={self._X: X_new_scaled})

        return y_pred

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#PID Tuning class    
class PIDTuning:
    def __init__(self, tau=None, zeta=None, theta=None, PID_type=None,
                 tau_modeling_error = 0, zeta_modeling_error = 0,
                 theta_modeling_error = 0, theta_tau_ratio_error = 0):
        self.tau = tau*(1+tau_modeling_error)
        self.zeta = zeta
        self.theta = theta*(1+tau_modeling_error)*(1+theta_tau_ratio_error)
        self.PID_type = PID_type

    def IMC_tuning_PIPC(self, process_gain=None):
        lamda = 0.25 * self.theta
        if self.PID_type == "PI":
            controller_gain = ((2*self.tau + self.theta)/2*lamda)/process_gain
            integral_time = self.tau + self.theta/2
            print("PI Controller by IMC Tuning_PIPC: Kc = %f, tau_i = %f" % (controller_gain, integral_time))
        elif self.PID_type == "PID":
            controller_gain = ((2*self.tau + self.theta)/2*(lamda + self.theta))/process_gain
            integral_time = self.tau + self.theta/2
            derivative_time = self.tau*self.theta/(2*self.tau + self.theta)
            print("PID Controller by IMC Tuning_PIPC: Kc = %f, tau_i = %f, tau_d = %f" %
                  (controller_gain, integral_time, derivative_time))
            
    def IMC_tuning_PDCT(self, process_gain=None, Condition=None):
        if Condition == "aggressive control":
            tau_c = 0.5/self.theta
        elif Condition == "default value":
            tau_c = self.theta
        elif Condition == "smooth control":
            tau_c = 1.5*self.theta
        elif Condition == "robust control":
            tau_c = 3*self.theta
        controller_gain = 2*self.tau*self.zeta/((tau_c+self.theta)*process_gain)
        integral_time = 2*self.tau*self.zeta
        derivative_time = self.tau/2*self.zeta
        print("PID Controller by IMC Tuning_PDCT: Kc = %f, tau_i = %f, tau_d = %f" %
             (controller_gain, integral_time, derivative_time))
    
    def ITAE_2_tuning(self, process_gain=None):
        theta_tau_ratio = self.theta/self.tau
        if self.zeta <= 0.9:
            controller_gain_set_point = (-0.04 + (0.333 + 0.949*(theta_tau_ratio**(-0.983)))*self.zeta)/process_gain
        else:
            controller_gain_set_point = (-0.544 + 0.308*theta_tau_ratio + 1.408*(theta_tau_ratio**(-0.832))*self.zeta)/process_gain
        if theta_tau_ratio <= 1.0:
            integral_time_set_point = (2.055 + 0.072*theta_tau_ratio)*self.zeta*self.tau
        else:
            integral_time_set_point = (1.768 + 0.329*theta_tau_ratio)*self.zeta*self.tau
        derivative_time_set_point = self.tau/((1.0 - np.exp(-(theta_tau_ratio**(1.060))*self.zeta/0.870))*\
        (0.55+1.683*(theta_tau_ratio**(-1.090))))
        if self.zeta <= 0.9:
            controller_gain_disturbance = (-0.670 + 0.297*(theta_tau_ratio**(-2.001)) +
                                           2.189*(theta_tau_ratio**(-0.766))*self.zeta)/process_gain
        else:    
            controller_gain_disturbance = (-0.365 + 0.260*((theta_tau_ratio - 1.400)**(2)) +
                                           2.189*(theta_tau_ratio**(-0.766))*self.zeta)/process_gain            
        if theta_tau_ratio < 0.4:
            integral_time_disturbance = (2.212*(theta_tau_ratio**(0.520)) - 0.300)*self.tau
        else:
            integral_time_disturbance = (-0.975 + 0.910*((theta_tau_ratio - 1.845)**(2)) +
                                         (1 - np.exp(-self.zeta/(0.150 + 0.330*theta_tau_ratio)))*
                                         (5.250 - 0.880*((theta_tau_ratio - 2.800)**(2))))
        derivative_time_disturbance = self.tau/(-1.900 + 1.576*(theta_tau_ratio**(-0.530)) +
                                                (1 - np.exp(-self.zeta/(-0.15 + 0.939*(theta_tau_ratio**(-1.121)))))*
                                                (1.45 + 0.969*(theta_tau_ratio**(-1.171))))
        print("PID Controller by ITAE-2-setpoint tuning: Kc = %f, tau_i = %f, tau_d = %f" %
              (controller_gain_set_point, integral_time_set_point, derivative_time_set_point))
        print("PID Controller by ITAE-2-disturbance tuning: Kc = %f, tau_i = %f, tau_d = %f" %
              (controller_gain_disturbance, integral_time_disturbance, derivative_time_disturbance))

In [3]:
#data load for training
learning_2nd = pd.read_excel('DataSecond_rand_limit_none_extra.xlsx')
learning_2nd_theta = pd.read_excel('DataSecond_rand_limit_extra.xlsx')
pro_iden = SecondOrderProcessIdentification(loaded_data=learning_2nd)
pro_iden_extra = SecondOrderProcessIdentification(n_inputs = 17, loaded_data=learning_2nd_theta)
X_train, y_train, X_test, y_test, X_scale_minmax = pro_iden.data_preparation()
y_train_TC, y_train_DC = y_train[:,0:1], y_train[:,1:2]
y_test_TC, y_test_DC = y_test[:,0:1], y_test[:,1:2]
X_train_theta, y_train_theta, X_test_theta, y_test_theta, X_scale_minmax_theta = pro_iden_extra.data_preparation_theta()
y_train_TD = y_train_theta[:,2:3]
y_test_TD = y_test_theta[:,2:3]

In [4]:
#Time Constant training
pro_iden_tau = SecondOrderProcessIdentification(n_hidden_layers=4, learning_rate = 0.00001, momentum = 0.999,
                                               learning_object="tau", random_state=42)
pro_iden_tau.fit(X=X_train, y=y_train_TC, n_epochs=52)
y_pred_TC = pro_iden_tau.predict(X=X_test, y=y_test_TC)
print("predictive time constant:\n", y_pred_TC)
print("real time constant:\n", y_test_TC[:10])

0	batch data accuracy: -3.7637	batch data RMSE: 16.484303
1	batch data accuracy: 0.5985	batch data RMSE: 6.905543
2	batch data accuracy: 0.7307	batch data RMSE: 4.382976
3	batch data accuracy: 0.8823	batch data RMSE: 1.760197
4	batch data accuracy: 0.9598	batch data RMSE: 0.735679
5	batch data accuracy: 0.9797	batch data RMSE: 0.426451
6	batch data accuracy: 0.9895	batch data RMSE: 0.311671
7	batch data accuracy: 0.9788	batch data RMSE: 0.284876
8	batch data accuracy: 0.9891	batch data RMSE: 0.210507
9	batch data accuracy: 0.9929	batch data RMSE: 0.195264
10	batch data accuracy: 0.9912	batch data RMSE: 0.166680
11	batch data accuracy: 0.9939	batch data RMSE: 0.160313
12	batch data accuracy: 0.9959	batch data RMSE: 0.141082
13	batch data accuracy: 0.9926	batch data RMSE: 0.112108
14	batch data accuracy: 0.9928	batch data RMSE: 0.150357
15	batch data accuracy: 0.9939	batch data RMSE: 0.153793
16	batch data accuracy: 0.9839	batch data RMSE: 0.134044
17	batch data accuracy: 0.9816	batch da

In [5]:
#Damping Coefficient training
pro_iden_zeta = SecondOrderProcessIdentification(n_hidden_layers=5, learning_rate = 0.00005,
                                                 momentum = 0.999, n_neurons = 140, batch_size = 150,
                                                 learning_object="zeta", random_state=42)
pro_iden_zeta.fit(X=X_train, y=y_train_DC, n_epochs=71)
y_pred_DC = pro_iden_zeta.predict(X=X_test, y=y_test_DC)
print("predictive damping coefficient:\n", y_pred_DC)
print("real damping coefficient:\n", y_test_DC[:10])

0	batch data accuracy: -0.0074	batch data RMSE: 0.254656
1	batch data accuracy: 0.8155	batch data RMSE: 0.108929
2	batch data accuracy: 0.7212	batch data RMSE: 0.068533
3	batch data accuracy: 0.8439	batch data RMSE: 0.050013
4	batch data accuracy: 0.8706	batch data RMSE: 0.029560
5	batch data accuracy: 0.9359	batch data RMSE: 0.012998
6	batch data accuracy: 0.9817	batch data RMSE: 0.007043
7	batch data accuracy: 0.9781	batch data RMSE: 0.007589
8	batch data accuracy: 0.9887	batch data RMSE: 0.004264
9	batch data accuracy: 0.9910	batch data RMSE: 0.003412
10	batch data accuracy: 0.9890	batch data RMSE: 0.003363
11	batch data accuracy: 0.9947	batch data RMSE: 0.003166
12	batch data accuracy: 0.9934	batch data RMSE: 0.001926
13	batch data accuracy: 0.9951	batch data RMSE: 0.001848
14	batch data accuracy: 0.9958	batch data RMSE: 0.001686
15	batch data accuracy: 0.9938	batch data RMSE: 0.001385
16	batch data accuracy: 0.9958	batch data RMSE: 0.001171
17	batch data accuracy: 0.9945	batch dat

In [6]:
#Time Delay training
pro_iden_theta = SecondOrderProcessIdentification(n_inputs=17, n_hidden_layers=4, learning_rate = 0.00001,
                                                  momentum = 0.999, n_neurons = 140, batch_size = 10,
                                                  learning_object="theta", random_state=42)
pro_iden_theta.fit(X=X_train_theta, y=y_train_TD, n_epochs=112)
y_pred_TD = pro_iden_theta.predict(X=X_test_theta, y=y_test_TD)
print("predictive time delay:\n", y_pred_TD)
print("real time delay:\n", y_test_TD[:10])

0	batch data accuracy: 0.9254	batch data RMSE: 0.643157
1	batch data accuracy: 0.9669	batch data RMSE: 0.073763
2	batch data accuracy: 0.9958	batch data RMSE: 0.062490
3	batch data accuracy: 0.9918	batch data RMSE: 0.066738
4	batch data accuracy: 0.9908	batch data RMSE: 0.095186
5	batch data accuracy: 0.9788	batch data RMSE: 0.044048
6	batch data accuracy: 0.9972	batch data RMSE: 0.022915
7	batch data accuracy: 0.9949	batch data RMSE: 0.042715
8	batch data accuracy: 0.9836	batch data RMSE: 0.045917
9	batch data accuracy: 0.9972	batch data RMSE: 0.037510
10	batch data accuracy: 0.9884	batch data RMSE: 0.059425
11	batch data accuracy: 0.9956	batch data RMSE: 0.045859
12	batch data accuracy: 0.9891	batch data RMSE: 0.033588
13	batch data accuracy: 0.9916	batch data RMSE: 0.041843
14	batch data accuracy: 0.9862	batch data RMSE: 0.052481
15	batch data accuracy: 0.9776	batch data RMSE: 0.018025
16	batch data accuracy: 0.9970	batch data RMSE: 0.018355
17	batch data accuracy: 0.9685	batch data

In [7]:
# data load for modeling
# The data used here is just data to work. Instead, change to the data you want to model.
operating_2nd = pd.read_excel('DataSecond_Operating_rand_limit_none_extra.xlsx')
operating_2nd_extra = pd.read_excel('DataSecond_Operating_rand_limit_extra.xlsx')
pro_iden = SecondOrderProcessIdentification(loaded_data = operating_2nd)
pro_iden_extra = SecondOrderProcessIdentification(n_inputs=17, loaded_data = operating_2nd_extra)
X_op, y_op, process_gain = pro_iden.operating_data_preparation(X_scale_minmax=X_scale_minmax)
y_op_TC, y_op_DC = y_op[:,0:1], y_op[:,1:2]
X_op_theta, y_op_theta = pro_iden_extra.operating_data_preparation_theta(X_scale_minmax=X_scale_minmax_theta)
y_op_TD = y_op_theta[:,2:3]

In [8]:
tau_pred = pro_iden_tau.operating_predict(X=X_op, y=y_op_TC)
zeta_pred = pro_iden_zeta.operating_predict(X=X_op, y=y_op_DC)
theta_pred = pro_iden_theta.operating_predict(X=X_op_theta, y=y_op_TD)

print("\n"); print("predictive time constant:", tau_pred)
print("\n"); print("predictive damping coefficient:", zeta_pred)
print("\n"); print("predictive time delay:", theta_pred)

INFO:tensorflow:Restoring parameters from ./2nd_order_process_tau_learning.ckpt
INFO:tensorflow:Restoring parameters from ./2nd_order_process_zeta_learning.ckpt
INFO:tensorflow:Restoring parameters from ./2nd_order_process_theta_learning.ckpt


predictive time constant: [[33.17033]]


predictive damping coefficient: [[0.09429336]]


predictive time delay: [[10.095656]]


In [9]:
PID_tuning = PIDTuning(tau=tau_pred, zeta=zeta_pred, theta=theta_pred, PID_type="PID")
process_gain = process_gain[:1]

PID_tuning.IMC_tuning_PIPC(process_gain=process_gain)
print("\n")
PID_tuning.IMC_tuning_PDCT(process_gain=process_gain, Condition="default value")
print("\n")
PID_tuning.ITAE_2_tuning(process_gain=process_gain)

PID Controller by IMC Tuning_PIPC: Kc = 16.745881, tau_i = 38.218159, tau_d = 4.381114


PID Controller by IMC Tuning_PDCT: Kc = 0.010757, tau_i = 6.255484, tau_d = 1.563871


PID Controller by ITAE-2-setpoint tuning: Kc = 0.009705, tau_i = 6.496050, tau_d = 163.562698
PID Controller by ITAE-2-disturbance tuning: Kc = 0.106017, tau_i = 29.576002, tau_d = 27.496960
